# mini hw3 - ensembles

In [45]:
import pandas as pd
import numpy as np
from google.colab import drive
from ipywidgets import interact
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from tqdm.notebook import tqdm
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.svm import SVC, SVR
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

## Подготовка данных

Загрузите и предобработайте данные (по своему усмотрению) из hw1

In [2]:
drive.mount('/content/gdrive/')
df = pd.read_csv('./gdrive/My Drive/Colab Notebooks/Homework 3/student-por.csv')

Mounted at /content/gdrive/


In [3]:
df.drop('G1', inplace = True, axis = 1)
df.drop('G2', inplace = True, axis = 1)
df.drop('Fedu', inplace = True, axis = 1)
df.drop('Walc', inplace = True, axis = 1)

In [4]:
for columns in df:
    print(columns, ':', df[columns].unique())

school : ['GP' 'MS']
sex : ['F' 'M']
age : [18 17 15 16 19 22 20 21]
address : ['U' 'R']
famsize : ['GT3' 'LE3']
Pstatus : ['A' 'T']
Medu : [4 1 3 2 0]
Mjob : ['at_home' 'health' 'other' 'services' 'teacher']
Fjob : ['teacher' 'other' 'services' 'health' 'at_home']
reason : ['course' 'other' 'home' 'reputation']
guardian : ['mother' 'father' 'other']
traveltime : [2 1 3 4]
studytime : [2 3 1 4]
failures : [0 3 1 2]
schoolsup : ['yes' 'no']
famsup : ['no' 'yes']
paid : ['no' 'yes']
activities : ['no' 'yes']
nursery : ['yes' 'no']
higher : ['yes' 'no']
internet : ['no' 'yes']
romantic : ['no' 'yes']
famrel : [4 5 3 1 2]
freetime : [3 2 4 1 5]
goout : [4 3 2 1 5]
Dalc : [1 2 5 3 4]
health : [3 5 1 2 4]
absences : [ 4  2  6  0 10  8 16 14  1 12 24 22 32 30 21 15  9 18 26  7 11  5 13  3]
G3 : [11 12 14 13 17 15  7 10 16  9  8 18  6  0  1  5 19]


In [5]:
types = df.dtypes
cat_cols = list(types[types == 'object'].index.values)
for col in cat_cols:
  df = pd.get_dummies(df, drop_first = True, columns = [col])

In [6]:
def counting(x):
  print('Nan values: ', df[x].isna().sum())
  print(df[x].value_counts())

interact(counting, x = [i for i in df])

interactive(children=(Dropdown(description='x', options=('age', 'Medu', 'traveltime', 'studytime', 'failures',…

<function __main__.counting(x)>

In [7]:
y = df['G3']
X = df.drop('G3', axis = 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

## Обоснуйте выбор слабых (базовых) алгоритмов

In [32]:
def get_regression_models():
	models = {
		"KNeighborsRegressor": KNeighborsRegressor(),
		"SVR": SVR(),
		"LinearRegression" : LinearRegression()
  }

	param_grid = {
  	"KNeighborsRegressor": {
      	"n_neighbors": np.arange(1, 25)
    },
    "SVR": {
    		"kernel": ["linear", "poly", "rbf"],
        "C": np.linspace(0.001, 10, 20)
    },
    "LinearRegression": {
			"fit_intercept": [True]
		}
  }

	meta_model_grid = {"kernel": ["linear", "poly", "rbf"], "C": np.linspace(0.001, 10, 20)}
	meta_model = SVR()

	return models, param_grid, meta_model, meta_model_grid

In [59]:
def get_classifier_models():
  models = {
    "KNeighborsClassifier": KNeighborsClassifier(),
    "SVC": SVC(probability=True),
    "LogisticRegression" : LogisticRegression()
  }
  
  param_grid = {
      "KNeighborsClassifier": {
          "n_neighbors": np.arange(1, 15)
      },
      "SVC": {
          "class_weight": ["balanced"],
          "kernel": ["linear", "poly", "rbf"],
          "C": np.linspace(0.001, 10, 20)
      },
      "LogisticRegression": {
          "solver": ['saga', 'lgbfs'],
          "C": [1, 2, 3],
          "penalty": ["l1","l2"],
          "max_iter": [100, 200]
      }
  }
  
  meta_model_grid = {"class_weight": ["balanced", "uniform"], "kernel": ["linear", "poly", "rbf"], "C": np.linspace(0.001, 10, 20)}
  meta_model = SVC()

  return models, param_grid, meta_model, meta_model_grid

## Постройте решение на основе подхода Blending

Правила:
- Нужно использовать вероятности
- Предложите что-то лучше, чем брать среднее от предсказаний моделей (оценивать уверенность алгоритмов, точности и т.д.)
- Заставьте базовые алгоритмы быть некорелированными
- Добавьте рандома (например, стройте ваши алгоритмы на разных выборках, по разному предобрабатывайте данные или применяйте для разных признаков соответствующие алгоритмы ... )
- Проявите смекалку

In [34]:
def fit_predict_CV(X, y, model_type, params_grid, verbose=False):
    model_cv = GridSearchCV(model_type, params_grid, cv = KFold(), refit = True)#, scoring='neg_mean_squared_error')
    model_cv.fit(X, y) 
    
    if verbose:
        print("Best hyperparameters: ", model_cv.best_params_)
        print("Best score: ", model_cv.best_score_)
    
    return model_cv.best_estimator_

In [35]:
def train_base_models(data, models, grid, verbose=False):
    for model_name in tqdm(models.keys()):
        if verbose:
            print(f"Model: {model_name}")
        models[model_name] = fit_predict_CV(data["X"], data["y"], models[model_name], grid[model_name], verbose=verbose)
        
    return models

In [36]:
def predict_base_models(X, models, models_type=None):
    if models_type is None:
        raise ValueError("models_type must be non None")
    predictions = None
    if models_type == "cls":
        for model in models.values():
            pred = model.predict_proba(X)
            if predictions is None:
                predictions = pred
            else:
                predictions = np.hstack([predictions, pred])
    if models_type == "reg":
         for model in models.values():
            pred = model.predict(X)
            if predictions is None:
                predictions = pred[:, None]
            else:
                predictions = np.hstack([predictions, pred[:, None]])
                
    return predictions

In [37]:
def train_meta_model(data, models, meta_model, grid, models_type=None, verbose=False):
    X_train_meta_model = predict_base_models(data["X"], models, models_type)
    return fit_predict_CV(X_train_meta_model, data["y"], meta_model, grid, verbose=verbose)

In [38]:
def predict_blending(data, models, meta_model, models_type=None):
    models_preds = predict_base_models(data["X"], models, models_type)
    predictions = meta_model.predict(models_preds)
    return predictions

In [39]:
X = df.drop(["G3"], axis=1)
y = df["G3"]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=42)

scaler = StandardScaler()
scaler.fit(x_train)

data_train = {
    "X": scaler.transform(x_train),
    "y": y_train
}
data_val = {
    "X": scaler.transform(x_val),
    "y": y_val
}
data_test = {
    "X": scaler.transform(x_test),
    "y": y_test
}

### Модели классификации

In [60]:
models, grid, meta_model, meta_model_grid = get_classifier_models()

In [61]:
best_models = train_base_models(data_train, models, grid, verbose=True)
trained_meta_model = train_meta_model(
    data_val, 
    best_models, 
    meta_model, 
    meta_model_grid,
    models_type="cls"
)

  0%|          | 0/3 [00:00<?, ?it/s]

Model: KNeighborsClassifier
Best hyperparameters:  {'n_neighbors': 12}
Best score:  0.15421686746987953
Model: SVC
Best hyperparameters:  {'C': 5.789894736842106, 'class_weight': 'balanced', 'kernel': 'poly'}
Best score:  0.19036144578313252
Model: LogisticRegression


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

Best hyperparameters:  {'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}
Best score:  0.1710843373493976


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
300 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py", line 199, in fit
    y = self._validate_targets(y)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py", line 718, in _validate_targets
    self.class_weight_ = compute_class_weight(self.class_weight, classes=cls, y=y_)
  File "/

In [62]:
predictions = np.round(predict_blending(data_test, best_models, trained_meta_model, models_type="cls"))
accuracy = accuracy_score(data_test["y"], np.round(predictions))
mse = mean_squared_error(data_test["y"], np.round(predictions))

print(f"Accuracy: {accuracy:.3}")
print(f"MSE:      {mse:.5}")

Accuracy: 0.154
MSE:      18.106


### Модели регрессии

In [43]:
models, grid, meta_model, meta_model_grid = get_regression_models()

In [44]:
best_models = train_base_models(data_train, models, grid, verbose=True)
trained_meta_model = train_meta_model(
    data_val, 
    best_models, 
    meta_model, 
    meta_model_grid,
    models_type="reg"
)

  0%|          | 0/3 [00:00<?, ?it/s]

Model: KNeighborsRegressor
Best hyperparameters:  {'n_neighbors': 19}
Best score:  0.1779615509588737
Model: SVR
Best hyperparameters:  {'C': 4.211105263157895, 'kernel': 'rbf'}
Best score:  0.27748303012982867
Model: LinearRegression
Best hyperparameters:  {'fit_intercept': True}
Best score:  0.21874837221986376


In [46]:
predictions = np.round(predict_blending(data_test, best_models, trained_meta_model, models_type="reg"))
accuracy = accuracy_score(data_test["y"], np.round(predictions))
mse = mean_squared_error(data_test["y"], np.round(predictions))

print(f"Accuracy: {accuracy:.3}")
print(f"MSE:      {mse:.5}")

Accuracy: 0.173
MSE:      5.7981


## Постройте решение на основе подхода Stacking

Правила:
- Реализуйте пайплайн обучения и предсказания (например, sklearn.pipeline или класс)
- Проведите оптимизацию пайплайна
- Оцените вклад каждого базового алгоритма в итоговое предсказание

In [47]:
models = get_classifier_models()[0]
estimators = []
for model_name in models.keys():
    model_pipeline = make_pipeline(
        StandardScaler(),
        models[model_name]
    )
    estimators += [(model_name, model_pipeline)]

folds = StratifiedKFold(n_splits=3, shuffle=True)

clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=folds
)

In [48]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

In [49]:
clf = clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

accuracy = accuracy_score(y_test, np.round(predictions))
mse = mean_squared_error(y_test, np.round(predictions))

print(f"Accuracy: {accuracy:.3}")
print(f"MSE:      {mse:.5}")

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:1092: RuntimeWarning: Number of classes in training fold (15) does not match total number of classes (16). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in proper

Accuracy: 0.192
MSE:      11.485


In [50]:
metrics = dict()
for exclude_model_name in models.keys():
    estimators = []
    for model_name in models.keys():
        if model_name != exclude_model_name:
            model_pipeline = make_pipeline(
                StandardScaler(),
                models[model_name]
            )
            estimators += [(model_name, model_pipeline)]
    
    folds = StratifiedKFold(n_splits=5, shuffle=True)

    clf = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression(),
        cv=folds
    )
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)
    clf = clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)

    accuracy = accuracy_score(y_test, np.round(predictions))
    mse = mean_squared_error(y_test, np.round(predictions))
    metrics[exclude_model_name] = (accuracy, mse)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:1092: RuntimeWarning: Number of classes in training fold (15) does not match total number of classes (16). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in proper

In [51]:
for model_name in models.keys():
    print("Metrics without", model_name)
    print(f"Accuracy: {metrics[model_name][0]:.3}")
    print(f"MSE:      {metrics[model_name][1]:.5}")
    print()

Metrics without KNeighborsClassifier
Accuracy: 0.2
MSE:      12.046

Metrics without SVC
Accuracy: 0.2
MSE:      11.169

Metrics without LogisticRegression
Accuracy: 0.169
MSE:      11.485

